In [1]:
from glob import glob
from os import path
import json
from pprint import pprint

glob('target-nightly-*')

def getIndividualBenchmark(targetDir, benchName):
    newDir = path.join(targetDir, 'criterion', benchName, 'new')
    mainEstimates = path.join(newDir, 'estimates.json')
    metricsEstimates = path.join(newDir, 'metrics-estimates.json')
    
    with open(metricsEstimates) as f:
        metrics = json.loads(f.read())
    with open(mainEstimates) as f:
        metrics['nanoseconds'] = json.loads(f.read())
    
    return metrics

def parseNightlyResult(targetDir):
    benches = glob(path.join(targetDir, 'criterion/*'))
    benchNames = [path.basename(b) for b in benches]
    allMetrics = {benchName: getIndividualBenchmark(targetDir, benchName) for benchName in benchNames}
    
    return allMetrics

rawResults = {tDir: parseNightlyResult(tDir) for tDir in glob('target-nightly-*')}

In [11]:
# transform results to be a series keyed by benchmark name
from datetime import datetime

results = {datetime.strptime(d.replace('target-nightly-', ''), '%Y-%m-%d'): v for d, v in rawResults.items()}

dates = set()
benches = set()
metricNames = set()
for date, metrics in results.items():
    dates.add(date)
    benches.update(metrics.keys())
    for _, metricName in metrics.items():
        metricNames.update(k for k in metricName.keys())

dates = sorted(list(dates))
benches = sorted(list(benches))
metricNames = sorted(list(metricNames))

In [12]:
import numbers

def descStatForGraphing(stat):
    forGraphing = {b: {m: dict() for m in metricNames} for b in benches}
    for bench in benches:
        for metricName in metricNames:
            estimatesAndErrors = []
            for date in dates:
                try:
                    measure = results[date][bench][metricName]
                    statistic = measure[stat]
                    ci95 = statistic['confidence_interval']
                    pointEstimate = statistic['point_estimate']
                    error = (ci95['upper_bound'] - ci95['lower_bound']) / 2

                    estimatesAndErrors.append((pointEstimate, error))
                except KeyError:
                    estimatesAndErrors.append((float('nan'), float('nan')))

            estimates, errors= zip(*estimatesAndErrors)
            forGraphing[bench][metricName] = { 
                'estimate': estimates, 
                'errors': errors 
            }
    
    with open('allresults-{}.json'.format(stat), 'w') as f:
        f.write(json.dumps(forGraphing))
    
    return forGraphing

meanForGraphing = descStatForGraphing('Mean')

In [24]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

def drawGraph(bench):
    plt.rcParams["figure.figsize"] = (20,100)
    plt.title(bench)
    for i, metricName in enumerate(metricNames):
        means = meanForGraphing[bench][metricName]['estimate'][7:30]
        meanErrors = meanForGraphing[bench][metricName]['errors'][7:30]
        
        plt.subplot(len(metricNames) / 2, 2, i+1)
        plt.title(metricName)
        plt.errorbar(x=dates[7:30], y=means, yerr=meanErrors, fmt='o')
        
    plt.show()
    
display(interactive(drawGraph, bench=benches))

aW50ZXJhY3RpdmUoY2hpbGRyZW49KERyb3Bkb3duKGRlc2NyaXB0aW9uPXUnYmVuY2gnLCBvcHRpb25zPSgnYnJvdGxpXzFfMV8zOjpiZW5jaF9lMmVfZGVjb2RlX3E1XzEwMjRrJywgJ2Jyb3TigKY=
